## Softmax Regression

이전 Logistic Regression은 두 가지 class의 구별을 학습하였다. 세 가지 이상일 때는 sigmoid로는 부족하기 때문에 다른 방법을 사용한다. <br /><br />
뭔가를 embedding vector로 표현하였다고 하자. X = (x1, x2, x3, x4) <br />
X를 어떤 함수에 통과시켜서 (c1, c2, c3) 이라는 벡터로 출력시키게 하는데, 이때 각 c1, c2, c3은 그 X가 class 1인지, 2인지, 3인지의 확률을 나타내도록 학습시키는 것이 바로 Softmax Regression이다. <br /><br />
정확히는 (c1, c2, c3) 이 출력 결과이면 softmax( (c1, c2, c3) ) = (p1, p2, p3) 이 된다.<br />
그러면 이제 생각해야 하는 것은 다음과 같다.<br />
 1. (x1, x2, x3, x4)를 (c1, c2, c3) 으로 변환시키는 방법 <br />
 2. (p1, p2, p3) 과 실제 정답 간의 오차를 계산하는 방법 <br />

1번의 해결책은 (4,3)짜리 행렬 W에 곱하고 (1,3) 벡터 b를 더하는 Linear Regression이다.<br />
2번의 해결책은, 각 정답 클래스를 one-hot encoding해서 이 둘 간의 오차를 cross-entropy로 계산하는 것이다. <br />
이 오차를 바탕으로 W의 각 성분을 학습해 나가게 된다.

In [1]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

In [30]:
# Cost function (Cross Entropy)

z = torch.rand(3,5,requires_grad=True)    # 3x5 tensor

# print(z)
hyp = F.softmax(z, dim=1)   # dim=1이라는 것은, z의 각 행을 한 개의 샘플로 본다는 뜻이다.
# print(hyp)                  # 세 개의 샘플 각각에 대해 5가지 클래스에 대한 확률이다.


y = torch.randint(5, (3,)).long()
y_one_hot = torch.zeros_like(hyp) 
y_one_hot.scatter_(1, y.unsqueeze(1), 1)    # one-hot vector 무작위로 생성

cost = (y_one_hot* -torch.log(hyp)).sum(dim=1).mean()     # 가장 low-level 구현
print(cost)

# softmax + log = F.log_softmax()
cost = (y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()
print(cost)

# F.nll_loss() 사용
cost = F.nll_loss(F.log_softmax(z, dim=1), y)
print(cost)

# F.cross_entropy() 사용
cost = F.cross_entropy(z, y)
print(cost)

tensor(1.5558, grad_fn=<MeanBackward0>)
tensor(1.5558, grad_fn=<MeanBackward0>)
tensor(1.5558, grad_fn=<NllLossBackward>)
tensor(1.5558, grad_fn=<NllLossBackward>)


# Softmax Regression 구현

In [33]:
#data
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

## 1. y_train을 one-hot vector로 encode <br />
8개의 샘플, 3개의 클래스이니 (8x3) 이 되어야 한다.

In [35]:
y_one_hot = torch.zeros(8,3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot)

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


## 2. Parameter들 정의하기
8x4 짜리 x_train에 W를 곱해서 (8,3)이 되어야 한다. 따라서 W는 (4,3) 텐서

In [39]:
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad = True)   # 나중에 broadcasting 된다.

optimizer = torch.optim.SGD([W,b], lr=0.1)

## 3. 나머지 부분 구현(Low-level)

In [43]:
nb_epochs = 10000
for epoch in range(nb_epochs) :
  hyp = F.softmax(x_train.matmul(W)+b, dim=1)
  cost = (y_one_hot*-torch.log(hyp)).sum(dim=1).mean()

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 2000 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/10000 Cost: 0.061307
Epoch 2000/10000 Cost: 0.052948
Epoch 4000/10000 Cost: 0.046560
Epoch 6000/10000 Cost: 0.041525
Epoch 8000/10000 Cost: 0.037459


## 3. 나머지 부분 구현(High-level)

In [45]:
nb_epochs = 10000
for epoch in range(nb_epochs) :

  z = x_train.matmul(W)+b
  cost = F.cross_entropy(z, y_train)
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 2000 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))


Epoch    0/10000 Cost: 0.034108
Epoch 2000/10000 Cost: 0.031302
Epoch 4000/10000 Cost: 0.028918
Epoch 6000/10000 Cost: 0.026868
Epoch 8000/10000 Cost: 0.025087


## 3. 나머지 부분 구현(nn.Module)

In [48]:
model = torch.nn.Linear(4,3)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 10000
for epoch in range(nb_epochs) :

  cost = F.cross_entropy(model(x_train), y_train)
  
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 2000 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))


Epoch    0/10000 Cost: 1.768461
Epoch 2000/10000 Cost: 0.160001
Epoch 4000/10000 Cost: 0.092196
Epoch 6000/10000 Cost: 0.064316
Epoch 8000/10000 Cost: 0.049256


## 3. 나머지 부분 구현 (Class)

In [56]:
class SoftmaxClassifier(torch.nn.Module) :
  def __init__(self) :
    super().__init__()
    self.linear = torch.nn.Linear(4,3)
  
  def forward(self, x) :
    return self.linear(x)

model = SoftmaxClassifier()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs) :
  prediction = model(x_train)
  cost = F.cross_entropy(prediction, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 200 == 0:
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/1000 Cost: 2.960336
Epoch  200/1000 Cost: 0.628710
Epoch  400/1000 Cost: 0.520265
Epoch  600/1000 Cost: 0.426433
Epoch  800/1000 Cost: 0.335029
